In [1]:
#Import necessary libraries
import os
import ee
import geemap
import ipyleaflet
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import statsmodels.api as sm
import pandas as pd
from IPython.display import HTML, display
import random
import json
import time
num_seed=30
random.seed(num_seed)

In [2]:
#Initialize earth engine
ee.Initialize()

#Read in collection of monthly average temperature in Kelvin
temp_collection= ee.ImageCollection('projects/resource-watch-gee/Facebook/TemperatureAnalysis/GHCN_CAMS')
temp_image = temp_collection.toBands()

#Get projection information for calculation
projection = temp_collection.first().projection().getInfo()
crs = projection.get('crs')
transform = projection.get('transform')

#Define folder for country boundaries
country_file_path = 'projects/resource-watch-gee/Facebook/TemperatureAnalysis/CountryShapes/{}'


In [3]:
#ADM0 (national level) calculation of monthly average temperature over each country
#Here we will loop through the countries in the country_list defined above, 
#calculate the monthly average temperature in the country's area
#and export to google drive.

#Define countries to loop over
country_list = ['FRA_adm0','DEU_adm0','GBR_adm0','USA_adm0','USA_adm0_contiguous']

#Create list of features to export
feature_list = []

#Loop through countries in list
for country in country_list:
    #Load country level polygon as Feature Collection
    adm0_file = country_file_path.format(country)
    adm0_feature = ee.FeatureCollection(adm0_file)
    
    #Calculate average temperature over region
    adm0_stats = temp_image.reduceRegion(
                                        reducer= ee.Reducer.mean(),
                                        geometry= adm0_feature.geometry(),
                                        crs = crs,
                                        crsTransform=transform,
                                        maxPixels= 1e13,
                                        bestEffort=True)
        
    #Update original country feature to include the average temperature statistics
    out_adm0_feature = ee.Feature(adm0_feature.first()).set(adm0_stats)
    #Add country name from country_list to identify all of US versus contiguous US
    out_adm0_feature = out_adm0_feature.set({'File Name':country}) 
    #Update geometry to None so we don't export it
    out_adm0_feature = out_adm0_feature.setGeometry(None)
    #Add to list of features to export
    feature_list.append(out_adm0_feature)

#Convert list of features to feature collection
feature_collection = ee.FeatureCollection(feature_list)

#Export results to Google Drive
export_adm0_stats = ee.batch.Export.table.toDrive(
    collection=feature_collection, 
    description = "adm0_kelvin", 
    fileNamePrefix = "adm0_kelvin")
export_adm0_stats.start()


In [4]:
#ADM1 (state level) calculation of monthly average temperature over each state within each country
#Here we will loop through the countries in the country_list defined above, 
#calculate the monthly average temperature in the country's state's 
#and export to google drive.
country_list = ['FRA_adm1','DEU_adm1','GBR_adm1','USA_adm1']

#Define list of attribute columns to keep
columns_to_keep = ['GID_0','HASC_1','NAME_0','NAME_1','VARNAME_1']

#Loop through countries in list
for country in country_list:
    #Load country level polygon as Feature Collection
    adm1_file = country_file_path.format(country)
    adm1_feature_collection = ee.FeatureCollection(adm1_file)
    #Select necessary columns
    adm1_feature_collection = adm1_feature_collection.select(columns_to_keep)
    
    #Get ist of states
    states = ee.List(adm1_feature_collection.aggregate_array('NAME_1'))
    
    def calculate_state_average_temp(state):
        '''
        Function for calculating the average monthly temperature within a given state
        '''
        states_collection = adm1_feature_collection.filterMetadata('NAME_1','equals',state)
        states_feature = ee.Feature(states_collection.first())
        adm1_stats = temp_image.reduceRegion(
                                            reducer= ee.Reducer.mean(),
                                            geometry= states_collection.geometry(),
                                            crs = crs,
                                            crsTransform=transform,
                                            maxPixels= 1e13,
                                            bestEffort=True)
        states_feature = states_feature.setGeometry(None)
        states_feature = states_feature.set(adm1_stats)
        return ee.Feature(states_feature)
    
    out_states_collection = ee.FeatureCollection(states.map(calculate_state_average_temp))
    export_adm1_stats = ee.batch.Export.table.toDrive(
        collection=out_states_collection, 
        description = "{}_kelvin".format(country), 
        fileNamePrefix = "{}_kelvin".format(country))
    export_adm1_stats.start()